In [1]:
import numpy as np
from cvxopt import solvers, matrix
solvers.options['show_progress'] = False

class SVM:
    
    def __init__(self, C=10.0, B=1.0, w_source=None):
        
        self.C = C                
        self.B = B
        if w_source is None:
            self.domain_adapt = False
        else:
            self.domain_adapt = True
        self.w_source = w_source  # source weights
        self.w = None             # weights
        self.b = None             # bias
        self.S = None             # indices of support vectors
        
    def fit(self, X, y):
        
        self.n, self.d = X.shape  # n: number of datapts, d: number of dims
        yy = y.reshape(-1, 1) * 1.
        X_dash = yy * X
        H = np.dot(X_dash , X_dash.T) * 1.
        
        P = matrix(H)
        if self.domain_adapt == False:
            q = matrix(-np.ones((self.n, 1)))
        else:
            extra_term = (self.B * y * (self.w_source.T @ X.T)).T
            q = matrix(-np.ones((self.n, 1)) + extra_term)
        G = matrix(np.vstack((-np.eye(self.n), np.eye(self.n))))
        h = matrix(np.hstack((np.zeros(self.n), np.ones(self.n) * self.C)))
        A = matrix(y.reshape(1, -1))
        b = matrix(np.zeros(1))
        
        sol = solvers.qp(P, q, G, h, A, b)
        alphas = np.array(sol['x'])
        
        self.w = ((yy * alphas).T @ X).reshape(-1, 1)
        self.S = (alphas > 1e-4).flatten()
        self.b = None  # I'll implement this later

In [2]:
source_data = np.genfromtxt('source_train.csv', delimiter=',')
target_data = np.genfromtxt('target_train.csv', delimiter=',')

X_s, y_s = source_data[:, 0: -1], source_data [:, -1]
X_t, y_t = target_data[:, 0: -1], target_data [:, -1]

In [3]:
C, B = 10., 1.
print(f'With C = {C} and B = {B}: \n')

clf = SVM(C=C)
clf.fit(X_s, y_s)
w_s = clf.w
print(f'w_source = ({w_s[0][0]:.5f}, {w_s[1][0]:.5f})')

clf = SVM(C=C, B=B, w_source=w_s)
clf.fit(X_t, y_t)
w_t = clf.w
print(f'w_target = ({w_t[0][0]:.5f}, {w_t[1][0]:.5f})')

With C = 10.0 and B = 1.0: 

w_source = (2.31500, 2.14257)
w_target = (1.80452, 0.78850)
